## Langchain Inference

In [1]:
!pip install langchain langchain-community bitsandbytes accelerate

Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 133.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.6 MB/s eta 0:00:00
Using cached nvidia_nvjitlink_cu12-12.6.20-py3-none-manylinux2014_x86_64.whl (19.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 404.1 kB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
from langchain_community.llms import HuggingFacePipeline, HuggingFaceHub

In [4]:
model_name = "RedHenLabs/news-reporter-3b"

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed,pipeline

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name,load_in_4bit=True, trust_remote_code=True, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/3.34k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/455 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [7]:
torch.cuda.empty_cache()

In [8]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512,
                temperature=0.1,
                return_full_text=False,
                do_sample=True)

## LCEL - Langchain Expression Language

In [29]:
hf = HuggingFacePipeline(pipeline=pipe)

### English - Inference

In [30]:
from langchain_core.prompts import PromptTemplate

template = """
<|user|>
Act as a news report and answer to the user question
Input: {question}
<|end|>
<|assistant|>
"""
prompt = PromptTemplate.from_template(template)

In [31]:
query = "What is the most common side effect of taking Chantix?"

In [32]:
chain = prompt | hf

In [33]:
print(chain.invoke({"question": query}))

 The most common side effect of taking Chantix is nausea, which occurs in about 10% of people. Other side effects include headache, dizziness, and insomnia.


### French Inference

In [34]:
fr_template = """
<|user|>
Act as a news report and answer to the user question. Both answer and question needs to be in French.
Input: {question}
<|end|>
<|assistant|>:
"""
fr_prompt = PromptTemplate.from_template(fr_template)

In [35]:
french_query = "Quel est l’effet secondaire le plus courant de la prise de Chantix"

In [36]:
fr_chain = fr_prompt | hf

In [37]:
print(fr_chain.invoke({"question": french_query}))


Question: Quel est l’effet secondaire le plus courant de la prise de Chantix?

Réponse: L'effet secondaire le plus courant de la prise de Chantix est la nausée, qui affecte environ 10% des patients.


### Spanish Inference

In [19]:
spa_template = """
<|user|>
Act as a news report and answer to the user question. Both answer and question needs to be in Spanish
Input: {question}
<|end|>
<|assistant|>
"""
spa_prompt = PromptTemplate.from_template(fr_template)

In [38]:
spa_query = "¿Cuál es el efecto de tomar Claritin durante 14 días?"

In [39]:
spa_chain = spa_prompt | hf

In [40]:
print(spa_chain.invoke({"question": spa_query}))


Question: ¿Cuál es el efecto de tomar Claritin durante 14 días?

Respuesta: Claritin, un antihistamínico, puede causar efectos secundarios como somnolencia, mareos, y diarrea. Es importante seguir las instrucciones del médico y no tomar más de lo indicado.


### German Inference with context example

In [41]:
deu_template = """
<|user|>
Act as a news report and answer to the user question from CONTEXT. Both answer and question needs to be in German. CONTEXT: {context}
Input: {question}
<|end|>
<|assistant|>
"""
deu_prompt = PromptTemplate.from_template(deu_template)

In [42]:
deu_query = "Wer ist Luke the Drifter und warum war er eine Entdeckung für Tom Hiddleston?"

In [25]:
deu_chain = deu_prompt | hf

In [27]:
context = """
Luke the Drifter war ein Pseudonym, unter dem Hank Williams gegen Ende seines Lebens Gedichte veröffentlichte, die Meditationen über die Kunst des Lebens, Einsamkeit, Traurigkeit und Tod waren. Tom Hiddleston empfand es als eine außergewöhnliche Entdeckung.
"""

In [28]:
print(deu_chain.invoke({"question": deu_query,"context":context}))

 Luke the Drifter war ein Pseudonym, unter dem Hank Williams Gedichte veröffentlichte, die tiefgründige Meditationen über das Leben, Einsamkeit, Traurigkeit und Tod waren. Tom Hiddleston empfand es als eine außergewöhnliche Entdeckung, da er die poetische Kraft und die emotionale Tiefe dieser Gedichte erkannte.


### Portuguese Inference with context example

In [43]:
por_template = """
<|user|>
Act as a news report and answer to the user question from Context. Both answer and question needs to be in Portuguese.  CONTEXT: {context}
Input: {question}
<|end|>
<|assistant|>
"""
por_prompt = PromptTemplate.from_template(por_template)

In [48]:
por_query = "Que medidas foram tomadas pelo Presidente Obama a 20 de Agosto de 2012, depois de o regime sírio ter ameaçado utilizar armas químicas?"

In [49]:
por_chain = por_prompt | hf

In [50]:
por_context = """
President Obama put Syrian leader Bashar al-Assad on notice by stating that the use of chemical weapons would be a red line for the US. He ordered the military to prepare to strike if Assad did indeed cross that red line.
"""

In [51]:
print(por_chain.invoke({"question": por_query,"context":por_context}))

 O Presidente Obama declarou que o uso de armas químicas por parte do regime sírio seria uma linha vermelha para os Estados Unidos. Ele ordenou que o exército se preparasse para uma resposta se essa linha fosse cruzada.
